In [30]:
#Import the packages we will need 
#Requests to interact with the api, pandas for working with the data, numpy for data calcs

import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json

In [31]:
#This function retrieves the gameweek used later in the code for filtering of the data


def get_recent_gameweek_id():
    """
    Get's the most recent gameweek's ID.
    """

    data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    data = json.loads(data.content)

    gameweeks = data['events']
    
    now = datetime.utcnow()
    for gameweek in gameweeks:
        next_deadline_date = datetime.strptime(gameweek['deadline_time'], '%Y-%m-%dT%H:%M:%SZ')
        if next_deadline_date > now:
            return gameweek['id'] - 1


if __name__ == '__main__':
     GW = float(get_recent_gameweek_id())
    
GW
    

31.0

In [53]:
#Initial download of all data from the fpl api

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

r = requests.get(url)

json = r.json()

#This lets us examine the JSON objext so we know what to do with it

json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [54]:
#Create pandas data frames for each of the data keys

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
elements_stats_df = pd.DataFrame(json['element_stats'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json['events'])
phases_df = pd.DataFrame(json['phases'])

#The columns for strength are misnamed so this funtion simply renames the columns.
teams_df.rename(columns={'strength_overall_home': 'strength_overall_away', 'strength_overall_away': 'strength_overall_home'}, inplace=True)

teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_away,strength_overall_home,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1200,1250,1130,1150,1220,1210,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1100,1160,1120,1140,1080,1100,2
2,36,0,None,3,0,Brighton,0,0,0,BHA,...,None,False,0,1130,1130,1150,1190,1100,1130,131
3,90,0,None,4,0,Burnley,0,0,0,BUR,...,None,False,0,1060,1070,1040,1090,1040,1080,43
4,8,0,None,5,0,Chelsea,0,0,0,CHE,...,None,False,0,1250,1280,1260,1300,1250,1310,4


In [34]:
#some unused charts


#fig = go.Figure([go.Bar(name="home", x=teams_df["name"], y= teams_df["strength_overall_home"],offsetgroup=0,),
 #                go.Bar(name ="away", x=teams_df["name"], y= teams_df["strength_overall_away"],offsetgroup=1,),],layout=go.Layout(
#        title="Overall strength"))              

#fig2 = go.Figure([go.Bar(name="home", x=teams_df["name"], y= teams_df["strength_attack_home"],offsetgroup=0,),
#                 go.Bar(name ="away", x=teams_df["name"], y= teams_df["strength_attack_away"],offsetgroup=1,),],layout=go.Layout(
#       title="Attacking strength"))

#fig3 = go.Figure([go.Bar(name="home", x=teams_df["name"], y= teams_df["strength_defence_home"],offsetgroup=0,),
 #                go.Bar(name ="away", x=teams_df["name"], y= teams_df["strength_defence_away"],offsetgroup=1,),],layout=go.Layout(
  #      title="Defensive strength"))

#fig.show()

#fig2.show()

#fig3.show()

In [35]:

#Filter out any columns we dont't need

slim_elements_df = elements_df[['first_name','second_name','team','element_type','minutes',
                                'now_cost','minutes','value_season','value_form','form','total_points','points_per_game',
                               'threat','influence','creativity','news']].copy()


#slim_elements_df.head(5)

In [36]:
slim_elements_df.loc[:,'position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
#slim_elements_df.head()

In [37]:
#Rename the team from the numerical index to the actual name 

slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)

#Change the data type from str to float to enable sorting 

slim_elements_df['value'] = slim_elements_df.value_season.astype(float)

slim_elements_df['form'] = slim_elements_df.value_season.astype(float)

#slim_elements_df['games'] = slim_elements_df.games.astype(float)

slim_elements_df['total_points'] = slim_elements_df.total_points.astype(float)

slim_elements_df.sort_values('value',ascending=False)

,first_name,second_name,team,element_type,minutes,now_cost,minutes,value_season,value_form,form,total_points,points_per_game,threat,influence,creativity,news,position,value
33,Emiliano,Martínez,Aston Villa,1,2700,54,2700,29.6,0.5,29.6,160.0,5.3,0.0,784.0,20.0,,Goalkeeper,29.6
324,Illan,Meslier,Leeds,1,2700,47,2700,27.9,1.1,27.9,131.0,4.4,0.0,852.4,10.0,,Goalkeeper,27.9
311,Stuart,Dallas,Leeds,2,2780,51,2780,27.1,1.2,27.1,138.0,4.5,386.0,593.0,357.9,,Defender,27.1
261,Alphonse,Areola,Fulham,1,2790,46,2790,24.6,0.3,24.6,113.0,3.6,0.0,723.0,0.0,,Goalkeeper,24.6
313,Patrick,Bamford,Leeds,4,2544,66,2544,24.5,0.7,24.5,162.0,5.2,1252.0,699.0,293.2,,Forward,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Sokratis,Papastathopoulos,Arsenal,2,0,48,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Left the club by mutual consent on 20/1,Defender,0.0
337,Gaetano,Berardi,Leeds,2,0,40,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,Defender,0.0
37,Neil,Taylor,Aston Villa,2,15,42,15,-0.2,0.0,-0.2,-1.0,-1.0,0.0,0.8,0.7,,Defender,-0.2
544,Alexandre,Jankewitz,Southampton,3,2,45,2,-0.2,0.0,-0.2,-1.0,-0.5,0.0,4.4,0.3,,Midfielder,-0.2


In [38]:

# Function for creating a column with whether the player is available or not
def unavailable(row):
    if row['news'] != '':
        return True
    else:
        return False
    

# Using function to create new column:
slim_elements_df['unavailable'] = slim_elements_df.apply(lambda row: unavailable(row), axis=1)

#slim_elements_df.head()

In [39]:
#Map the team strength from the teamd data frame to the elements data frame

slim_elements_df['strength_overall_home'] = slim_elements_df.team.map(teams_df.set_index('name')['strength_overall_home'].to_dict())

slim_elements_df['strength_overall_away'] = slim_elements_df.team.map(teams_df.set_index('name')['strength_overall_away'].to_dict())

slim_elements_df.head()

,first_name,second_name,team,element_type,minutes,now_cost,minutes,value_season,value_form,form,...,points_per_game,threat,influence,creativity,news,position,value,unavailable,strength_overall_home,strength_overall_away
0,Mesut,Özil,Arsenal,3,0,67,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,Not included in Arsenal's 25-man Premier Leagu...,Midfielder,0.0,True,1250,1200
1,Sokratis,Papastathopoulos,Arsenal,2,0,48,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,Left the club by mutual consent on 20/1,Defender,0.0,True,1250,1200
2,David,Luiz Moreira Marinho,Arsenal,2,1344,54,1344,7.4,0.1,7.4,...,2.1,123.0,244.6,45.2,Knee injury - Unknown return date,Defender,7.4,True,1250,1200
3,Pierre-Emerick,Aubameyang,Arsenal,3,2007,114,2007,9.5,0.1,9.5,...,4.3,731.0,428.2,292.4,,Midfielder,9.5,False,1250,1200
4,Cédric,Soares,Arsenal,2,744,46,744,6.1,0.1,6.1,...,2.8,66.0,110.8,114.8,,Defender,6.1,False,1250,1200


In [40]:

#Code block for grouping data into positional data frames and plotting them for visual analysis 


grouped_se_df = slim_elements_df.groupby(slim_elements_df.position)

df_gk = grouped_se_df.get_group("Goalkeeper").copy()
df_df = grouped_se_df.get_group("Defender").copy()
df_mfd = grouped_se_df.get_group("Midfielder").copy()
df_fwd = grouped_se_df.get_group("Forward").copy()

df_gk.sort_values(['value'],inplace =True)
df_df.sort_values(['value'],inplace =True)
df_mfd.sort_values(['value'],inplace =True)
df_fwd.sort_values(['value'],inplace =True)


fig_ovr_val = px.scatter(slim_elements_df, x="now_cost", y="total_points", hover_data=['second_name'])


fig_gk = px.bar(df_gk, x = 'second_name', y = 'value',hover_data = ['team'])
fig_df = px.bar(df_df, x = 'second_name', y = 'value',hover_data = ['team'])
fig_mfd = px.bar(df_mfd, x = 'second_name', y = 'value',hover_data = ['team'])
fig_fwd = px.bar(df_fwd, x = 'second_name', y = 'value',hover_data = ['team'])

fig_gk_tp = px.scatter(df_gk, x="second_name", y="total_points", hover_data=['now_cost'])
fig_df_tp = px.scatter(df_df, x="second_name", y="total_points", hover_data=['now_cost'])
fig_mfd_tp = px.scatter(df_mfd, x="second_name", y="total_points", hover_data=['now_cost'])
fig_fwd_tp = px.scatter(df_fwd, x="second_name", y="total_points", hover_data=['now_cost'])

fig_gk.add_trace(fig_gk_tp.data[0])
fig_df.add_trace(fig_df_tp.data[0])
fig_mfd.add_trace(fig_mfd_tp.data[0])
fig_fwd.add_trace(fig_fwd_tp.data[0])

fig_ovr_val.show()

fig_gk.show()
fig_df.show()
fig_mfd.show()
fig_fwd.show()

In [41]:

#Defining a dict to allow us to determine the home or away team 

Venues = {'Team':  ['West Brom','Sheffield Utd','Burnley','Southampton',
             'Fulham','Newcastle','Brighton','Crystal Palace',
            'Leeds','Wolves','Aston Villa','Arsenal',
             'Spurs','West Ham','Leicester','Everton',
            'Liverpool','Chelsea','Man Utd','Man City'],
        'Ground': ['The Hawthorns','Bramall Lane','Turf Moor',"St. Mary's Stadium",
                  'Craven Cottage',"St James' Park", 'Amex Stadium', 'Selhurst Park',
                   'Elland Road','Molineux Stadium','Villa Park','Emirates Stadium',
                  'Tottenham Hotspur Stadium','London Stadium','King Power Stadium',
                  'Goodison Park','Anfield','Stamford Bridge','Old Trafford','Etihad Stadium']}

Venues_df = pd.DataFrame (Venues, columns = ['Team','Ground']) #Converting the dict to a pd data frame

#Venues_df.head()

In [51]:
#Importing fixtures and filtering for just the game week at hand using the predefined gameweek variable


filename = "epl-2020-GMTStandardTime.csv"

fixtures = pd.read_csv("/Users/jamesdawson/Desktop/Data analysis /Python/" + filename) 

fixtures_filtered = fixtures.loc[fixtures['Round Number'] == (GW)].copy()

fixtures_filtered.head()



,Round Number,Date,Location,Home Team,Away Team,Result
301,31,09/04/2021 20:00,Craven Cottage,Fulham,Wolves,NaN
302,31,10/04/2021 12:30,Etihad Stadium,Man City,Leeds,NaN
303,31,10/04/2021 15:00,Anfield,Liverpool,Aston Villa,NaN
304,31,10/04/2021 17:30,Selhurst Park,Crystal Palace,Chelsea,NaN
305,31,11/04/2021 12:00,Turf Moor,Burnley,Newcastle,NaN


In [43]:
#A slightly convoluted block but essentially just takes the fixtures, duplicates the table and flips the home and away team
#then does an if the team in column a matches column b then the team is home if not away. In essence transform the 
#team home/away colum into a value in each row.



fixtures_flipped = fixtures.copy()

fixtures_flipped.rename(columns={'Home Team': 'Away Team', 'Away Team': 'Home Team'}, inplace=True)

fixtures_flipped_filtered = fixtures_flipped.loc[fixtures_flipped['Round Number'] == (GW)].copy()


combined_fixtures = fixtures_filtered.append(fixtures_flipped_filtered, ignore_index=True)


combined_fixtures['Ground'] = combined_fixtures['Home Team'].map(Venues_df.set_index('Team')['Ground'].to_dict())


def f(row):
    if row['Location'] == row['Ground']:
        val = 'Home'
    else:
        val = 'Away'
    return val


combined_fixtures['Home/Away'] = combined_fixtures.apply(f, axis=1)


In [44]:
#Maps on opponent, home/away strength anf calcs the home advantage


slim_elements_df['opponent'] = slim_elements_df.team.map(combined_fixtures.set_index('Home Team')['Away Team'].to_dict())

slim_elements_df['HomeAway'] = slim_elements_df.team.map(combined_fixtures.set_index('Home Team')['Home/Away'].to_dict())

slim_elements_df['opponent_hso'] = slim_elements_df.opponent.map(teams_df.set_index('name')['strength_overall_home'].to_dict())

slim_elements_df['opponent_aso'] = slim_elements_df.opponent.map(teams_df.set_index('name')['strength_overall_away'].to_dict())


slim_elements_df['HomeAdv'] = np.where(slim_elements_df['HomeAway'] == 'Home',
                                        slim_elements_df['strength_overall_home']/slim_elements_df['opponent_aso'],
                                        slim_elements_df['strength_overall_away']/slim_elements_df['opponent_hso'])


#turn the form into a factor

slim_elements_df['form_adj'] = (1 / (100 -(slim_elements_df['form'])) +1)

#define adjusted value

slim_elements_df['Value_Adjusted'] = slim_elements_df['value'] * slim_elements_df['HomeAdv'] * slim_elements_df['form_adj']

slim_elements_df[200:210]

,first_name,second_name,team,element_type,minutes,now_cost,minutes,value_season,value_form,form,...,unavailable,strength_overall_home,strength_overall_away,opponent,HomeAway,opponent_hso,opponent_aso,HomeAdv,form_adj,Value_Adjusted
200,Fabian,Delph,Everton,3,211,48,211,1.2,0.0,1.2,...,True,1130,1250,Brighton,Away,1130,1130,1.106195,1.010121,1.340869
201,Gylfi,Sigurdsson,Everton,3,1555,68,1555,12.9,0.0,12.9,...,False,1130,1250,Brighton,Away,1130,1130,1.106195,1.011481,14.433745
202,Jonas,Lössl,Everton,1,0,43,0,0.0,0.0,0.0,...,True,1130,1250,Brighton,Away,1130,1130,1.106195,1.010000,0.000000
203,Seamus,Coleman,Everton,2,973,48,973,9.0,0.8,9.0,...,False,1130,1250,Brighton,Away,1130,1130,1.106195,1.010989,10.065156
204,Cenk,Tosun,Everton,4,44,60,44,0.7,0.0,0.7,...,True,1130,1250,Brighton,Away,1130,1130,1.106195,1.010070,0.782134
205,Bernard,Anício Caldeira Duarte,Everton,3,390,57,390,3.2,0.0,3.2,...,True,1130,1250,Brighton,Away,1130,1130,1.106195,1.010331,3.576391
206,Lucas,Digne,Everton,2,1871,62,1871,12.3,0.1,12.3,...,False,1130,1250,Brighton,Away,1130,1130,1.106195,1.011403,13.761339
207,Michael,Keane,Everton,2,2359,51,2359,19.6,0.2,19.6,...,False,1130,1250,Brighton,Away,1130,1130,1.106195,1.012438,21.951085
208,Jordan,Pickford,Everton,1,2022,48,2022,16.5,0.0,16.5,...,True,1130,1250,Brighton,Away,1130,1130,1.106195,1.011976,18.470802
209,André Filipe,Tavares Gomes,Everton,3,1386,53,1386,7.5,0.2,7.5,...,True,1130,1250,Brighton,Away,1130,1130,1.106195,1.010811,8.386152


In [45]:
#Split the df for stars and sort by total points

slim_elements_df_stars = slim_elements_df.copy()

slim_elements_df_stars.sort_values(['total_points'],inplace = True, ascending = False)

In [46]:
#slim_elements_df_stars.head(5)

In [47]:
#split the df for best value adjusted players

slim_elements_df_roi = slim_elements_df.copy()

slim_elements_df_roi.sort_values(['Value_Adjusted'], inplace = True, ascending = False)

slim_elements_df_roi.head()

,first_name,second_name,team,element_type,minutes,now_cost,minutes,value_season,value_form,form,...,unavailable,strength_overall_home,strength_overall_away,opponent,HomeAway,opponent_hso,opponent_aso,HomeAdv,form_adj,Value_Adjusted
33,Emiliano,Martínez,Aston Villa,1,2700,54,2700,29.6,0.5,29.6,...,False,1160,1100,Liverpool,Away,1210,1250,0.909091,1.014205,27.291322
324,Illan,Meslier,Leeds,1,2700,47,2700,27.9,1.1,27.9,...,False,1170,1160,Man City,Away,1360,1340,0.852941,1.013870,24.127115
311,Stuart,Dallas,Leeds,2,2780,51,2780,27.1,1.2,27.1,...,False,1170,1160,Man City,Away,1360,1340,0.852941,1.013717,23.431780
261,Alphonse,Areola,Fulham,1,2790,46,2790,24.6,0.3,24.6,...,False,1100,1090,Wolves,Home,1110,1080,1.018519,1.013263,25.387857
313,Patrick,Bamford,Leeds,4,2544,66,2544,24.5,0.7,24.5,...,False,1170,1160,Man City,Away,1360,1340,0.852941,1.013245,21.173841


In [48]:
#Theoretical best team 


def team_selecter():
    
    budget = 1000

    position_dict = {"Goalkeeper":2,"Defender":5,"Midfielder": 5, "Forward": 3}

    team_dict = {'West Brom':3,'Sheffield Utd':3,'Burnley':3,'Southampton':3,
             'Fulham':3,'Newcastle':3,'Brighton':3,'Crystal Palace':3,
            'Leeds':3,'Wolves':3,'Aston Villa':3,'Arsenal':3,
             'Spurs':3,'West Ham':3,'Leicester':3,'Everton':3,
            'Liverpool':3,'Chelsea':3,'Man Utd':3,'Man City':3}

    roi_team = [] #form an empty tuple for the team
    
    total_points = 0
    
    top_performer_limit = 3
    
    # Choosing 3 top performers from the "top players"-dataframe
    for idx, row in slim_elements_df_stars.iterrows():
        if budget >= row.now_cost and len(roi_team) < top_performer_limit and row.unavailable == False and position_dict[row.position] != 0 and team_dict[row.team] != 0:
            roi_team.append(row.second_name)
            budget -= row.now_cost #Deducting cost from budget
            position_dict[row.position] -= 1 # Deducting position from position dictionary
            team_dict[row.team] -= 1 # Deducting player from team dictionary
            total_points += row.total_points # adding to point score
            print("Player choosen from 'top players' " + str(row.second_name))
            
                   
        else:
            for idx, row in slim_elements_df_roi.iterrows():
                if row.second_name not in roi_team and budget >= row.now_cost and row.unavailable == False and position_dict[row.position] != 0 and team_dict[row.team] != 0:
                    roi_team.append(row.second_name)
                    budget -= row.now_cost
                    position_dict[row.position] -= 1 # Deducting position from position dictionary
                    team_dict[row.team] -= 1 # Deducting player from team dictionary
                    total_points += row.total_points # adding to point score
    
                   
    print("\nTeam chosen: " + str(roi_team))
    print("Remaining budget: " + str((budget/10)) + "M.")
    print("Total points from choosen team: " + str(total_points) + ".")
    print(position_dict)

In [55]:
#team_selecter()